### 获得在线数据

In [ ]:
import os
import pendulum
from impala.dbapi import connect
import numpy as np
import pandas as pd
import jinja2
os.environ.update(
        {
            "HOST": "bbx.lilithgame.com",
            "PORT": "10000",
            "API_KEY": "9954b1025d9abd6f31b4a3cb6b190a9830Uop36Pa5m",
            "API_SECRET": "UPM9rnL6tJ9G80l6Ci6ZcjdS55ZizCHTW3G0Wvx5Ks",
            "_MAX_RETRY_TIMES": "5",
            "_TASK_CONFIG_FILE": "config/rok_revenue_festival_time.toml",
        }
    )
config = dict(
            host=os.environ.get("HOST") or "bbx.lilithgame.com",
            port=os.environ.get("PORT") or 10000,
            user=os.environ.get("API_KEY"),
            password=os.environ.get("API_SECRET"),
            auth_mechanism="plain",
            timeout=100)

In [2]:
def _result_iterator(cursor, buffer=1000):
    while True:
        results = cursor.fetchmany(buffer)
        if not results:
            break
        for result in results:
            yield result
def execute_pandas(cursor):
    res = [_r for _r in _result_iterator(cursor, 1000)]
    col = [r[0] for r in cursor.description]
    return pd.DataFrame(res, columns=col)
def get_cursor():
    conn = connect(**config)
    cursor = conn.cursor()
    engine_type = "trino"
    cursor.execute(f"set lilith.dispatcher.exec.engine.type={engine_type}")
    return cursor

In [3]:
with open('analysis.sql') as f:
    value = f.read()
f.close()

In [4]:
df_ls = []
dt_ls = ['2023-11-1'+str(i) for i in range(0,10)] + ['2023-11-2'+str(i) for i in range(0,10)] + ['2023-11-30'] + ['2023-12-0'+str(i) for i in range(1,10)]
#2023.11.10 ~ 2023.11.19 + 2023.11.20 ~ 2023.11.29+ 2023.11.30 + 2023.12.01 ~2023.12.09
for dt in dt_ls:
    print(dt)
    cursor = get_cursor()
    cursor.execute(value.format(ds = dt))
    df = execute_pandas(cursor)
    df_ls.append(df)
    cursor.close()

2023-11-10
2023-11-11
2023-11-12
2023-11-13
2023-11-14
2023-11-15
2023-11-16
2023-11-17
2023-11-18
2023-11-19
2023-11-20
2023-11-21
2023-11-22
2023-11-23
2023-11-24
2023-11-25
2023-11-26
2023-11-27
2023-11-28
2023-11-29
2023-11-30
2023-12-01
2023-12-02
2023-12-03
2023-12-04
2023-12-05
2023-12-06
2023-12-07
2023-12-08
2023-12-09


In [5]:
data = pd.concat(df_ls,axis=0)
data['bundle_price'] = data['bundle_price'].astype('float')
data['profit'] = data['bundle_price'] * data['bundle_buy_cnt']/100 #单位上美分，所以/100
data_star = data[(data['st_type']=='star_with_attention') | (data['st_type']=='star0828') | (data['st_type']=='control')]

In [6]:
data_star.to_csv('analysis_30day.csv',index=False)

### 分析实验组，对照组的收益指标（弹出次数，购买次数，购买率，单次弹出收益）

In [7]:
def analysis(df,groupname):
    result = df.groupby(groupname).agg({'bundle_trigger_cnt':'sum','bundle_buy_cnt':'sum','profit':'sum'}).reset_index()
    result['购买率'] = result['bundle_buy_cnt']/result['bundle_trigger_cnt']
    result['单次弹出收益'] = result['profit']/result['bundle_trigger_cnt']
    
    control = result[result['st_type']=='control'][[groupname[1],'bundle_trigger_cnt','bundle_buy_cnt','profit','购买率','单次弹出收益']]
    control.columns = [groupname[1],'对照组_弹出次数','对照组_购买次数','对照组_profit','对照组_购买率','对照组_单次弹出收益']
    
    star = result[result['st_type']=='star0828'][[groupname[1],'bundle_trigger_cnt','bundle_buy_cnt','profit','购买率','单次弹出收益']]
    star.columns = [groupname[1],'star_弹出次数','star_购买次数','star_profit','star_购买率','star_单次弹出收益']
    
    attention_star = result[result['st_type']=='star_with_attention'][[groupname[1],'bundle_trigger_cnt','bundle_buy_cnt','profit','购买率','单次弹出收益']]
    attention_star.columns = [groupname[1],'attention_弹出次数','attention_购买次数','attention_profit','attention_购买率','attention_单次弹出收益']
    
    x = pd.merge(control,star,on=groupname[1],how='outer')
    x = pd.merge(x,attention_star,on=groupname[1],how='outer')
    
    return x

#### 分trigger

In [8]:
analysis(data_star,['st_type','trigger_type'])

,trigger_type,对照组_弹出次数,对照组_购买次数,对照组_profit,对照组_购买率,对照组_单次弹出收益,star_弹出次数,star_购买次数,star_profit,star_购买率,star_单次弹出收益,attention_弹出次数,attention_购买次数,attention_profit,attention_购买率,attention_单次弹出收益
0,advancecard_unlock_trigger_air_force,951.0,29.0,2899.71,0.030494,3.049117,16934,377,29356.23,0.022263,1.733567,4352,112,8568.88,0.025735,1.968952
1,advancecard_unlock_trigger_ground_force,4439.0,68.0,6799.32,0.015319,1.531723,73616,1675,110313.25,0.022753,1.498496,18183,384,26196.16,0.021119,1.440695
2,blueprint_max_trigger_air_force,1334.0,7.0,349.93,0.005247,0.262316,28330,351,28896.49,0.012390,1.019996,7023,84,7519.16,0.011961,1.070648
3,blueprint_max_trigger_ground_force,7901.0,48.0,2399.52,0.006075,0.303698,149973,1305,89731.95,0.008702,0.598321,37409,331,24736.69,0.008848,0.661250
4,gvg_trigger,23803.0,107.0,2408.93,0.004495,0.101203,296518,1289,28042.11,0.004347,0.094571,73788,314,7076.86,0.004255,0.095908
5,population_consume_trigger,10453.0,195.0,9748.05,0.018655,0.932560,158353,3473,203925.27,0.021932,1.287789,39195,944,53780.56,0.024085,1.372128
6,blackmall_trigger,NaN,NaN,NaN,NaN,NaN,260612,4241,249487.59,0.016273,0.957314,66273,1048,66689.52,0.015813,1.006285


#### 分礼包价格

In [9]:
analysis(data_star,['st_type','bundle_price'])

,bundle_price,对照组_弹出次数,对照组_购买次数,对照组_profit,对照组_购买率,对照组_单次弹出收益,star_弹出次数,star_购买次数,star_profit,star_购买率,star_单次弹出收益,attention_弹出次数,attention_购买次数,attention_profit,attention_购买率,attention_单次弹出收益
0,999.0,5582.0,23.0,229.77,0.004120,0.041163,224159,747,7462.53,0.003332,0.033291,56323,154,1538.46,0.002734,0.027315
1,1499.0,1776.0,18.0,269.82,0.010135,0.151926,24558,228,3417.72,0.009284,0.139169,5947,46,689.54,0.007735,0.115948
2,1999.0,1864.0,7.0,139.93,0.003755,0.075070,286195,3877,77501.23,0.013547,0.270799,71023,968,19350.32,0.013629,0.272451
3,2999.0,14581.0,59.0,1769.41,0.004046,0.121350,183013,770,23092.30,0.004207,0.126178,45628,187,5608.13,0.004098,0.122910
4,4999.0,19688.0,250.0,12497.50,0.012698,0.634778,32030,566,28294.34,0.017671,0.883370,8111,148,7398.52,0.018247,0.912159
5,9999.0,5390.0,97.0,9699.03,0.017996,1.799449,77561,5973,597240.27,0.077010,7.700265,19596,1594,159384.06,0.081343,8.133500
6,499.0,NaN,NaN,NaN,NaN,NaN,156820,550,2744.50,0.003507,0.017501,39595,120,598.80,0.003031,0.015123


#### 分创角时常

In [10]:
analysis(data_star,['st_type','create_days_type'])

,create_days_type,对照组_弹出次数,对照组_购买次数,对照组_profit,对照组_购买率,对照组_单次弹出收益,star_弹出次数,star_购买次数,star_profit,star_购买率,star_单次弹出收益,attention_弹出次数,attention_购买次数,attention_profit,attention_购买率,attention_单次弹出收益
0,0-7,75,2,29.98,0.026667,0.399733,7047,128,3078.72,0.018164,0.436884,1811,34,849.66,0.018774,0.469166
1,121-150,2574,18,859.82,0.006993,0.334040,31687,447,27300.53,0.014107,0.861569,8032,69,4939.31,0.008591,0.614954
2,15-30,4471,36,1159.64,0.008052,0.259369,72582,593,14479.07,0.008170,0.199486,18198,110,2648.90,0.006045,0.145560
3,151-180,1785,47,2299.53,0.026331,1.288252,24921,385,24146.15,0.015449,0.968908,5993,94,5719.06,0.015685,0.954290
4,181-240,5798,91,4639.09,0.015695,0.800119,55888,807,47451.93,0.014440,0.849054,14204,234,13802.66,0.016474,0.971745
5,241-360,3696,34,2279.66,0.009199,0.616791,112517,1842,119141.58,0.016371,1.058876,27766,487,31155.13,0.017539,1.122060
6,31-60,2204,24,739.76,0.010889,0.335644,48597,603,23133.97,0.012408,0.476037,11657,129,4438.71,0.011066,0.380776
7,360+,21529,172,11598.28,0.007989,0.538728,530405,6779,432882.21,0.012781,0.816135,133001,1791,116187.09,0.013466,0.873581
8,61-90,3256,11,279.89,0.003378,0.085961,53887,525,19539.75,0.009743,0.362606,13681,126,5998.74,0.009210,0.438472
9,8-14,955,11,269.89,0.011518,0.282607,12757,155,3843.45,0.012150,0.301282,3258,20,269.80,0.006139,0.082812


#### 分充值等级

In [11]:
analysis(data_star,['st_type','recharge_group'])

,recharge_group,对照组_弹出次数,对照组_购买次数,对照组_profit,对照组_购买率,对照组_单次弹出收益,star_弹出次数,star_购买次数,star_profit,star_购买率,star_单次弹出收益,attention_弹出次数,attention_购买次数,attention_profit,attention_购买率,attention_单次弹出收益
0,0,17616,2,59.98,0.000114,0.003405,271215,204,12177.96,0.000752,0.044901,67022,58,3809.42,0.000865,0.056838
1,0-100,12286,9,99.91,0.000733,0.008132,252504,399,6026.01,0.001580,0.023865,62774,76,849.24,0.001211,0.013529
2,100-500,6251,11,199.89,0.001760,0.031977,137249,776,14807.24,0.005654,0.107886,34575,194,3588.06,0.005611,0.103776
3,10000+,2389,302,18096.98,0.126413,7.575128,70234,6687,520303.13,0.095210,7.408138,18189,1725,139722.75,0.094838,7.681717
4,2000-10000,4985,103,5208.97,0.020662,1.044929,125409,3137,143098.63,0.025014,1.141056,31776,843,38796.57,0.026529,1.220939
5,500-2000,5354,27,939.73,0.005043,0.175519,127725,1508,43339.92,0.011807,0.339322,31887,321,7801.79,0.010067,0.244670


#### 分国服/国际服

In [12]:
analysis(data_star,['st_type','svr_region'])

,svr_region,对照组_弹出次数,对照组_购买次数,对照组_profit,对照组_购买率,对照组_单次弹出收益,star_弹出次数,star_购买次数,star_profit,star_购买率,star_单次弹出收益,attention_弹出次数,attention_购买次数,attention_profit,attention_购买率,attention_单次弹出收益
0,global,36319,306,17826.94,0.008425,0.490843,969859.0,12685.0,738143.15,0.013079,0.761083,242611.0,3199.0,192938.01,0.013186,0.795257
1,ios_ptr,596,0,0.00,0.000000,0.000000,14477.0,26.0,1609.74,0.001796,0.111193,3612.0,18.0,1629.82,0.004983,0.451224
2,official,11966,148,6778.52,0.012368,0.566482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
#data.to_csv('analysis.csv',index=False)